In [38]:
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_base")
from gliner_spacy.pipeline import GlinerSpacy
import spacy
from spacy import displacy
from spacy.matcher import PhraseMatcher

In [ ]:
## Step 1: GLiNER

In [41]:
text = """
Nutrition labs at the next visit
Follow nutrition recommendations.
Encouraged intake of 3 meals per day of low protein foods.
Encouraged intake of water instead of calorie drinks.
Aim for <15gm per day of protein from diet
Discussed importance of daily diet log to aid with protein counting and weight loss
Begin vitamin D supplementation, 1000 IUs per day 
We will contact the PCPs office and request assistance in placing referral to a local therapist/psychologist
We will continue to work on obtaining insurance coverage for her formula. 
If this continues to be deferred, we will consider starting tyrosine supplementation in the interim.
Followup in the metabolic clinic in 1 months  
It was a pleasure to see Renato Shin. 
"""

labels= ['breast milk','protein', 'supplementation'
    'liquid predominant diet',
    'formula', 
    'supplementation',
    'meals',
    'water']

entities = model.predict_entities(text, labels, threshold=0.5)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

water => breast milk
calorie drinks => formula
vitamin D => supplementation
formula => formula
tyrosine supplementation => supplementation


## Step 2: Example with spacy

In [56]:
nlp = spacy.load("en_core_web_sm")
phrase_matcher = PhraseMatcher(nlp.vocab)
food_list = [nlp.make_doc(text) for text in [
  'breast milk','protein', 'supplementation',
    'liquid predominant diet',
    'formula', 
    'supplementation',
    'meals',
    'water']]
phrase_matcher.add("DIET",None, *food_list)
doc1    = nlp(text)
matches = phrase_matcher(doc1)
# Assign labels and update Doc object with entities
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the label
    span = doc1[start : end]  # get the matched slice of the doc
    print(rule_id,"=>", span.text)
 

DIET => meals
DIET => protein
DIET => water
DIET => protein
DIET => protein
DIET => supplementation
DIET => formula
DIET => supplementation


In [54]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY
